In [71]:
#find snps
#import data science libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import allel
import bokeh
import iqplot
import numpy as np
import scipy.stats as st
#suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [72]:
root = "/Users/bencekover/Library/CloudStorage/OneDrive-Personal/MSci Bahler lab/S.-Pombe-MLPs/"
snps_file = '/Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/Jeffares copy/Jeffares SNPs/ERZ289255/Spombe.2013-01-02.filt3c.nr57-final.snps.anno-snpeff3.cleaned4.vcf'
indels_file = '/Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/Jeffares copy/Jeffares SNPs/ERZ289254/Spombe.indels.unique.nr57.final.annotated.2014-01-23.cleaned4.vcf.gz'
curation_file = root + "Bence folder/Looking for variants in wild isolates/bence gene curation.txt"
deletion_screen_file = root + "Bence folder/Angeli/bioneer_angeli_95thpercentile.txt"





In [73]:
#genome coordinates for genes
df = pd.read_csv('/Users/bencekover/Library/CloudStorage/OneDrive-Personal/MSci Bahler lab/S.-Pombe-MLPs/external data/Pombase files/Schizosaccharomyces_pombe_all_chromosomes.gff3', sep='\t', header=1)
#break last column into 2 at ;
df[['a','b']] = df["ID=SPAC1002.01;Name=mrx11"].str.split(';',expand=True)
#remove column ID=SPAC1002.01;Name=mrx11
df.drop(columns =["ID=SPAC1002.01;Name=mrx11"], inplace = True)
#remove ID= from column a and rename it to ID
df['ID'] = df['a'].str.replace('ID=', '')
#remove column a
df.drop(columns =['a','PomBase',".","..1"], inplace = True)
#rename b to Parent
df.rename(columns={'b':'Parent','gene':'object','I':'chromosome','1798347':"start",'1798835':"end","+":"strand"}, inplace=True)
#In parent remove anything before =
df['Parent'] = df['Parent'].str.replace('Parent=', '')
#or Name=
df['Parent'] = df['Parent'].str.replace('Name=', '')
#create an aempty column called detail
df['detail'] = ""
#split ID column at :, and if there is something after put it in detail column
df['detail'] = df['ID'].str.split(':', expand=True)[1]
#split ID column at :, and if there is something before put it in ID column
df['ID'] = df['ID'].str.split(':', expand=True)[0]#load in vcf file

In [74]:
df = df.loc[df["object"] == "gene"]
#mediator srb10, srb11, med12 and med13
curation = pd.read_csv(curation_file, sep='\t', header=0)["Systematic ID"]
mediator = ["SPAC23H4.17c","SPBC12D12.06","SPAC688.08","SPAC589.02c","SPBC15D4.02"]
deletion_screen_genes = pd.read_csv(deletion_screen_file, sep='\t', header=0)["gene"]

df = df.loc[(df["ID"].isin(mediator) | df["ID"].isin(curation) | df["ID"].isin(deletion_screen_genes))]
df.reset_index(drop=True, inplace=True)
df.shape

(180, 8)

In [75]:
#interesting_strains = samples
interesting_strains = ["JB914","JB871","JB1117", "JB872", "JB934", "JB759", "JB848"]

In [76]:
snps_data = allel.read_vcf(snps_file,fields=['samples','variants/CHROM','variants/POS','calldata/GT','variants/ALT','variants/REF'])
samples = snps_data['samples']
#finding the index of the interesting strains
interesting_strains_index = []
for i in range(len(samples)):
      if samples[i] in interesting_strains:
            interesting_strains_index.append(i)

calls= snps_data['calldata/GT'][:,:,0]
chromosomes = snps_data['variants/CHROM']
positions = snps_data['variants/POS']
alt = snps_data['variants/ALT']
ref= snps_data['variants/REF']


In [77]:
#iterate through positions
#results is a df with columns: alleles	n	sum_x	y_transpose_x	beta	standard_error	t_stat	p_value	type	allele1	allele2	chromosome	position
results = pd.DataFrame(columns=["strain","ID","chromosome","position","gene","type","allele1","allele2","p_value"])
#rough gene coordinates
#take mean of start and end
rough_gene_coordinates = (df["start"]+df["end"])/2


In [78]:

for i in range(len(positions)):
      #if positions[i] is not near to any of the rough_gene_coordinates  +-10000
      
      if np.isclose(positions[i],rough_gene_coordinates,atol=20000).any() == False:
            
            continue
      for j in range(len(df)):
            #for + strand
            if positions[i]<(df["end"][j]+200) and positions[i]>(df["start"][j]-200) and chromosomes[i]==df["chromosome"][j]:
                  #iterate through interesting strains
                  for k in range(len(interesting_strains_index)):
                        #take the value of the genotype of the interesting strain at the position
                        call_value = calls[i][interesting_strains_index[k]]
                        #if this value is 0 or 1, and there is only max 2 strains with this value, then add the strain name to the detail column
                        if call_value == 0 or call_value == 1 or call_value == 2:
                              if sum(calls[i]==call_value) <= 5:
                                    
                                    results = results.append({"strain":interesting_strains[k],"ID":df["ID"][j],"chromosome":chromosomes[i],"position":positions[i], "gene":df["Parent"][j],"type":"snp","allele1":ref[i],"allele2":alt[i][call_value-1],"p_value":0.05},ignore_index=True)
                              


In [79]:
indels_data = allel.read_vcf(indels_file,fields=['samples','variants/CHROM','variants/POS','calldata/GT','variants/ALT','variants/REF'])

calls= indels_data['calldata/GT'][:,:,0]
chromosomes = indels_data['variants/CHROM']
positions = indels_data['variants/POS']
alt = indels_data['variants/ALT']
ref= indels_data['variants/REF']

for i in range(len(positions)):
   
      if np.isclose(positions[i],rough_gene_coordinates,atol=2000).any() == False:
            continue
      for j in range(len(df)):
            #for + strand
            if positions[i]<(df["end"][j]+200) and positions[i]>(df["start"][j]-200) and chromosomes[i]==df["chromosome"][j]:
                  #iterate through interesting strains
                  for k in range(len(interesting_strains_index)):
                        #take the value of the genotype of the interesting strain at the position
                        call_value = calls[i][interesting_strains_index[k]]
                        #if this value is 0 or 1, and there is only max 3 strains with this value, then add the strain name to the detail column
                        if call_value == 0 or call_value == 1 or call_value == 2 or call_value == 3:
                              if sum(calls[i]==call_value) <= 3:
                                    
                                    results = results.append({"strain":interesting_strains[k],"ID":df["ID"][j],"chromosome":chromosomes[i],"position":positions[i], "gene":df["Parent"][j],"type":"indel","allele1":ref[i],"allele2":alt[i][call_value-1],"p_value":0.05},ignore_index=True)
                              


In [82]:
#make csv file
results.to_csv(root + "Bence folder/Looking for variants in wild isolates/snp_hunting.csv", index=False)